# ⏱️ Building a Word-Counter Video with VideoDB

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/examples/Keyword_Search_Counter`.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
With an endless stream of new video content on our feeds, engaging the audience with dynamic visual elements can make educational and promotional videos much more impactful. VideoDB's suite of features allows you to enhance videos with programmatic editing.

In this tutorial, we'll explore how to create a video that visually counts and displays instances of a specified word as it's spoken. We'll use VideoDB’s [Keyword Search ](https://docs.videodb.io)to index spoken words, and then apply audio and [text overlays](https://docs.videodb.io) to show a counter updating in real-time with synchronized audio cues.

## Setup
---


### 📦  Installing packages 

In [ ]:
%pip install videodb

### 🔑 API Keys
Before proceeding, ensure access to [VideoDB](https://videodb.io) API key. 

> Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required** ) 🎉

In [22]:
import os

os.environ["VIDEO_DB_API_KEY"] = ""

## Steps
---

### 🔗  Step 1: Connect to VideoDB

In [23]:
import videodb

conn = videodb.connect()
coll = conn.get_collection()

### 🎥 Step 2: Upload Video
Upload and play the video to ensure it's correctly loaded. We’ll be using [this video](https://www.youtube.com/watch?v=Js4rTM2Z1Eg) for the purpose of this tutorial.

In [24]:
video = coll.upload(url="https://www.youtube.com/watch?v=Js4rTM2Z1Eg")
video.play()

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/09b866a6-ccf8-4d11-b37c-0a2b9edc78aa.m3u8'

### 📝 Step 3:  Indexing Spoken Words
Index the video to identify and timestamp all spoken words.

In [25]:
video.index_spoken_words()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:34<00:00,  2.92it/s]


### 🔍  Step 4:  Keyword Search

Search within the video for the keyword ("education" in this example), and note each occurrence.


In [26]:
from videodb import SearchType

result = video.search(query='education', search_type=SearchType.keyword)

### 🎼  Step 5:  Setup Timeline 

Initialize the timeline and retrieve an audio asset to use for each word occurrence.

In [31]:
from videodb.timeline import Timeline
from videodb.asset import AudioAsset
from videodb import MediaType

timeline = Timeline(conn)


audio = conn.upload(url="", media_type=MediaType.audio)

audio_asset = AudioAsset(asset_id=audio.id,
    start=0,
    end=1.7,
    disable_other_tracks=False,
    fade_in_duration=1,
    fade_out_duration=0,
)

### 💬 Step 6:  Overlay Text and Audio

Add text and audio overlays at each instance where the word is spoken.

>
>Note: Adding the ‘padding’ is an optional step. It helps in adding a little more context to the exact instance identified, thus resulting in a better compiled output.
>

In [32]:
from videodb.asset import TextAsset, TextStyle, VideoAsset, AudioAsset

seeker = 0
counter = 0
padding = 1.5

for shot in result.shots:
    duration = shot.end - shot.start + 2 * padding
    
    # VideoAsset for each Shot
    video_asset = VideoAsset(
        asset_id=shot.video_id, start=shot.start - padding, end=shot.end + padding
    )

    # TextAsset that displays count
    text_asset = TextAsset(
        text=f"Count-{counter}",
        duration=duration,
        style=TextStyle(
            font="Do Hyeon",
            fontsize=70,
            fontcolor="#000100",
            box=True,
            boxcolor="F702A4",
            x=900,
            y=80,
        ),
    )


    timeline.add_inline(asset=video_asset)
    timeline.add_overlay(asset=text_asset, start=seeker - padding)
    timeline.add_overlay(asset=audio_asset, start=seeker + padding)

    seeker += duration
    counter += 1

### ⚡️ Step 7: Generate and Play the Stream

Finally, generate a streaming URL for your edited video and play it.

In [33]:
from videodb import play_stream

stream_url = timeline.generate_stream()
play_stream(stream_url)


'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/de823df2-b6a8-4384-937b-77767d666c99.m3u8'

## Conclusion
---

This tutorial showcases VideoDB's capabilities to create a video that programmatically counts and displays the frequency of a specific keyword spoken throughout the video. This method can be adapted for various applications where dynamic text overlays add significant value to video content.

### Tips and Tricks

* Use different text styles and positions based on your video's theme.
* Add background sounds or effects to enhance the viewer's experience.